In [ ]:
!pip install pymupdf
!pip install gradio
!pip install openai-whisper
!pip install soundfile
!pip install gtts
!pip install nest_asyncio
!pip install langchain-text-splitters
!pip install langchain-huggingface
!pip install langchain-community
!pip install langchain-groq
!pip install faiss-cpu
!pip install transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 12.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=644ece798af1b4c8e2f374bf3cf694d138eb674961c9215536571306ef12eaa6
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1

In [ ]:
# --- IMPORTS ---
import fitz  # (this is PyMuPDF)
import os
import gradio as gr
import whisper
import soundfile as sf
from gtts import gTTS
import nest_asyncio
nest_asyncio.apply()

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate


In [ ]:
# --- CONFIG ---
os.environ["GROQ_API_KEY"] = ""
HUGGINGFACE_TOKEN = "your_huggingface_token"    # replace if needed


# --- MODELS ---
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
embeddings = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
whisper_model = whisper.load_model("base")

prompt = PromptTemplate(
    template="""
You are a helpful AI assistant that answers based only on the PDF context below.
If you don’t know the answer, say “I don’t know from this page.”

Context:
{context}

Question:
{question}
""",
    input_variables=["context", "question"]
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 227MiB/s]


In [ ]:
# ---------------------------------------------------------
#  Helper: Create / Load page vectorstore (lazy loading)
# ---------------------------------------------------------
def get_or_create_page_vector(page_num):
    global PDF_NAME, PDF_PATH

    if PDF_NAME is None or PDF_PATH is None:
        return None, None

    page_dir = f"vector_dbs/{PDF_NAME}/page_{page_num+1}"

    try:
        if os.path.exists(page_dir):
            store = FAISS.load_local(
                page_dir, embeddings, allow_dangerous_deserialization=True
            )
            pdf = fitz.open(PDF_PATH)
            text = pdf.load_page(page_num).get_text("text").strip()
            pdf.close()
            return store, text

        pdf = fitz.open(PDF_PATH)
        if page_num < 0 or page_num >= pdf.page_count:
            pdf.close()
            return None, None

        page = pdf.load_page(page_num)
        text = page.get_text("text").strip()
        pdf.close()

        if not text:
            return None, ""

        docs = splitter.create_documents([text])
        os.makedirs(page_dir, exist_ok=True)

        store = FAISS.from_documents(docs, embeddings)
        store.save_local(page_dir)

        return store, text

    except Exception as e:
        print("get_or_create_page_vector error:", e)
        return None, None


# ---------------------------------------------------------
#  Chat with current page
# ---------------------------------------------------------
def chat_with_pdf_current_page(question, page_num):
    store, text_or_msg = get_or_create_page_vector(page_num)

    if not store:
        return text_or_msg or "❌ No text/vector available.", ""

    retriever = store.as_retriever(search_type="mmr", search_kwargs={"k": 3})
    docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in docs)

    final_prompt = prompt.invoke({"context": context, "question": question})
    answer = llm.invoke(final_prompt)

    best_chunk = docs[0].page_content if docs else ""
    return answer.content, best_chunk


# ---------------------------------------------------------
#  Multi-page RAG
# ---------------------------------------------------------
def multi_page_rag(question, page_from, page_to):
    try:
        page_from = int(page_from)
        page_to = int(page_to)
    except:
        return "⚠️ Invalid page range.", ""

    if page_from < 0 or page_to < page_from:
        return "⚠️ Invalid page range.", ""

    collected_docs = []

    for p in range(page_from, page_to + 1):
        store, text = get_or_create_page_vector(p)
        if store:
            retriever = store.as_retriever(search_type="mmr", search_kwargs={"k": 2})
            docs = retriever.invoke(question)
            if docs:
                collected_docs.extend(docs)

    if not collected_docs:
        return "⚠️ No relevant content found.", ""

    seen = set()
    unique_docs = []
    for d in collected_docs:
        txt = d.page_content.strip()
        if txt not in seen:
            seen.add(txt)
            unique_docs.append(d)

    top_docs = unique_docs[:6]
    context = "\n\n".join(d.page_content for d in top_docs)

    final_prompt = prompt.invoke({"context": context, "question": question})
    answer = llm.invoke(final_prompt)

    best_chunk = top_docs[0].page_content if top_docs else ""
    return answer.content, best_chunk


# ---------------------------------------------------------
#  Highlight text
# ---------------------------------------------------------
def highlight_text(full_text, matched_chunk):
    if not matched_chunk or not full_text:
        return full_text or ""

    safe_chunk = matched_chunk.strip().replace("\n", " ")

    if safe_chunk in full_text:
        return full_text.replace(safe_chunk, f"<mark>{safe_chunk}</mark>")

    small = safe_chunk[:100].strip()
    if small and small in full_text:
        return full_text.replace(small, f"<mark>{small}</mark>")

    return full_text


# ---------------------------------------------------------
#  Compare pages
# ---------------------------------------------------------
def compare_pages(page_a, page_b):
    try:
        page_a = int(page_a)
        page_b = int(page_b)
    except:
        return "⚠️ Invalid page numbers."

    store_a, text_a = get_or_create_page_vector(page_a)
    store_b, text_b = get_or_create_page_vector(page_b)

    if not store_a or not store_b:
        return "⚠️ One or both pages have no text."

    compare_prompt = f"""
Compare these two pages:

--- Page {page_a+1} ---
{text_a}

--- Page {page_b+1} ---
{text_b}

Give:
1. Key similarities
2. Key differences
3. Critical insights
"""

    result = llm.invoke(compare_prompt)
    return result.content


# ---------------------------------------------------------
#  Process PDF Upload
# ---------------------------------------------------------
def process_pdf(file):
    global PDF_NAME, PDF_PATH, TOTAL_PAGES

    PDF_PATH = file.name
    PDF_NAME = os.path.splitext(os.path.basename(file.name))[0]

    pdf = fitz.open(PDF_PATH)
    TOTAL_PAGES = pdf.page_count
    pdf.close()

    os.makedirs(f"vector_dbs/{PDF_NAME}", exist_ok=True)

    return (
        f"✅ '{PDF_NAME}' loaded ({TOTAL_PAGES} pages).",
        "",
        gr.update(value=0, minimum=0, maximum=max(0, TOTAL_PAGES - 1), step=1)
    )


# ---------------------------------------------------------
#  Update page display
# ---------------------------------------------------------
def update_page(page_num):
    try:
        page_num = int(page_num)
    except:
        return "⚠️ Invalid page index.", ""

    store, text = get_or_create_page_vector(page_num)

    if store is None and text is None:
        return "⚠️ No PDF loaded.", ""

    return f"📖 Page {page_num+1} ready.", text or ""


def update_page_with_highlight(page_num, matched_chunk):
    status, text = update_page(page_num)
    highlighted = highlight_text(text, matched_chunk)
    return status, highlighted


# ---------------------------------------------------------
#  Voice query (current page)
# ---------------------------------------------------------
def voice_query(audio, page_num):
    try:
        result = whisper_model.transcribe(audio)
        text = result.get("text", "")

        if WAKE_WORD not in text.lower():
            return f"⚠️ Wake word not detected. Say '{WAKE_WORD}'.", None

        query = text.lower().replace(WAKE_WORD, "").strip()

        answer, best_chunk = chat_with_pdf_current_page(query, int(page_num))

        output_file = "response.mp3"
        if os.path.exists(output_file):
            os.remove(output_file)

        tts = gTTS(answer)
        tts.save(output_file)

        return f"🎤 You said: {query}\n\n🤖 {answer}", output_file

    except Exception as e:
        return f"❌ Error: {e}", None


In [ ]:

# ---------------------------------------------------------
#                      GRADIO UI
# ---------------------------------------------------------
with gr.Blocks(theme="soft") as demo:

    gr.Markdown("## 📚 Advanced PDF RAG (Page, Range, Compare, Voice)")

    pdf_upload = gr.File(label="📄 Upload PDF")
    status = gr.Textbox(label="Status", interactive=False)
    pdf_text = gr.HTML("<i>Page text will appear here</i>")

    page_slider = gr.Slider(label="Page", minimum=0, maximum=0, step=1, value=0)

    with gr.Row():
        page_from = gr.Number(label="From Page", value=0)
        page_to = gr.Number(label="To Page", value=0)

    question = gr.Textbox(label="Ask Question")
    answer = gr.Textbox(label="Answer", lines=6)
    matched_chunk = gr.Textbox(visible=False)

    ask_current_btn = gr.Button("Ask Current Page")
    ask_range_btn = gr.Button("Ask Page Range")

    with gr.Row():
        compare_page_a = gr.Number(label="Page A", value=0)
        compare_page_b = gr.Number(label="Page B", value=1)
    compare_btn = gr.Button("🔍 Compare Pages")
    compare_output = gr.Textbox(label="Comparison", lines=6)

    audio_in = gr.Audio(label="🎙️ Speak Query", type="filepath")
    audio_out = gr.Audio(label="🔊 Audio Reply")

    # Handlers
    pdf_upload.change(process_pdf, inputs=pdf_upload, outputs=[status, pdf_text, page_slider])
    page_slider.change(update_page_with_highlight, inputs=[page_slider, matched_chunk],
                       outputs=[status, pdf_text])

    ask_current_btn.click(
        lambda q, p: chat_with_pdf_current_page(q, int(p)),
        inputs=[question, page_slider],
        outputs=[answer, matched_chunk]
    )

    ask_range_btn.click(
        lambda q, pf, pt: multi_page_rag(q, pf, pt),
        inputs=[question, page_from, page_to],
        outputs=[answer, matched_chunk]
    )

    compare_btn.click(
        compare_pages,
        inputs=[compare_page_a, compare_page_b],
        outputs=compare_output
    )

    audio_in.change(
        voice_query,
        inputs=[audio_in, page_slider],
        outputs=[answer, audio_out]
    )

demo.launch(share=True, debug=True, inline=False, prevent_thread_lock=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7db7237139fb8f0715.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1623, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

FULL NEW CODE


In [ ]:
# --- IMPORTS ---
import fitz
import os
import gradio as gr
import whisper
import soundfile as sf
from gtts import gTTS
import nest_asyncio
nest_asyncio.apply()

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate


# --- CONFIG ---
os.environ["GROQ_API_KEY"] = "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"
HUGGINGFACE_TOKEN = "your_huggingface_token"    # replace if needed


# --- MODELS ---
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
embeddings = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
whisper_model = whisper.load_model("base")

prompt = PromptTemplate(
    template="""
You are a helpful AI assistant that answers based only on the PDF context below.
If you don’t know the answer, say “I don’t know from this page.”

Context:
{context}

Question:
{question}
""",
    input_variables=["context", "question"]
)


# --- GLOBALS ---
PDF_NAME = None
PDF_PATH = None
TOTAL_PAGES = 0
WAKE_WORD = "hey"


# ---------------------------------------------------------
#  Helper: Create / Load page vectorstore (lazy loading)
# ---------------------------------------------------------
def get_or_create_page_vector(page_num):
    global PDF_NAME, PDF_PATH

    if PDF_NAME is None or PDF_PATH is None:
        return None, None

    page_dir = f"vector_dbs/{PDF_NAME}/page_{page_num+1}"

    try:
        # Load stored FAISS vector index
        if os.path.exists(page_dir):
            store = FAISS.load_local(
                page_dir, embeddings, allow_dangerous_deserialization=True
            )
            pdf = fitz.open(PDF_PATH)
            text = pdf.load_page(page_num).get_text("text").strip()
            pdf.close()
            return store, text

        # Create new vector index
        pdf = fitz.open(PDF_PATH)
        if page_num < 0 or page_num >= pdf.page_count:
            pdf.close()
            return None, None

        page = pdf.load_page(page_num)
        text = page.get_text("text").strip()
        pdf.close()

        if not text:
            return None, ""

        docs = splitter.create_documents([text])
        os.makedirs(page_dir, exist_ok=True)

        store = FAISS.from_documents(docs, embeddings)
        store.save_local(page_dir)

        return store, text

    except Exception as e:
        print("get_or_create_page_vector error:", e)
        return None, None


# ---------------------------------------------------------
#  Chat with current page
# ---------------------------------------------------------
def chat_with_pdf_current_page(question, page_num):
    store, text_or_msg = get_or_create_page_vector(page_num)

    if not store:
        return text_or_msg or "❌ No text/vector available.", ""

    retriever = store.as_retriever(search_type="mmr", search_kwargs={"k": 3})
    docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in docs)

    final_prompt = prompt.invoke({"context": context, "question": question})
    answer = llm.invoke(final_prompt)

    best_chunk = docs[0].page_content if docs else ""
    return answer.content, best_chunk


# ---------------------------------------------------------
#  Multi-page RAG
# ---------------------------------------------------------
def multi_page_rag(question, page_from, page_to):
    try:
        page_from = int(page_from)
        page_to = int(page_to)
    except:
        return "⚠️ Invalid page range.", ""

    if page_from < 0 or page_to < page_from:
        return "⚠️ Invalid page range.", ""

    collected_docs = []

    for p in range(page_from, page_to + 1):
        store, text = get_or_create_page_vector(p)
        if store:
            retriever = store.as_retriever(search_type="mmr", search_kwargs={"k": 2})
            docs = retriever.invoke(question)
            if docs:
                collected_docs.extend(docs)

    if not collected_docs:
        return "⚠️ No relevant content found.", ""

    # Deduplicate
    seen = set()
    unique_docs = []
    for d in collected_docs:
        txt = d.page_content.strip()
        if txt not in seen:
            seen.add(txt)
            unique_docs.append(d)

    top_docs = unique_docs[:6]
    context = "\n\n".join(d.page_content for d in top_docs)

    final_prompt = prompt.invoke({"context": context, "question": question})
    answer = llm.invoke(final_prompt)

    best_chunk = top_docs[0].page_content if top_docs else ""
    return answer.content, best_chunk


# ---------------------------------------------------------
#  Highlight text
# ---------------------------------------------------------
def highlight_text(full_text, matched_chunk):
    if not matched_chunk or not full_text:
        return full_text or ""

    safe_chunk = matched_chunk.strip().replace("\n", " ")

    if safe_chunk in full_text:
        return full_text.replace(safe_chunk, f"<mark>{safe_chunk}</mark>")

    small = safe_chunk[:100].strip()
    if small and small in full_text:
        return full_text.replace(small, f"<mark>{small}</mark>")

    return full_text


# ---------------------------------------------------------
#  Compare pages
# ---------------------------------------------------------
def compare_pages(page_a, page_b):
    try:
        page_a = int(page_a)
        page_b = int(page_b)
    except:
        return "⚠️ Invalid page numbers."

    store_a, text_a = get_or_create_page_vector(page_a)
    store_b, text_b = get_or_create_page_vector(page_b)

    if not store_a or not store_b:
        return "⚠️ One or both pages have no text."

    compare_prompt = f"""
Compare these two pages:

--- Page {page_a+1} ---
{text_a}

--- Page {page_b+1} ---
{text_b}

Give:
1. Key similarities
2. Key differences
3. Critical insights
"""

    result = llm.invoke(compare_prompt)
    return result.content


# ---------------------------------------------------------
#  Process PDF Upload
# ---------------------------------------------------------
def process_pdf(file):
    global PDF_NAME, PDF_PATH, TOTAL_PAGES

    PDF_PATH = file.name
    PDF_NAME = os.path.splitext(os.path.basename(file.name))[0]

    pdf = fitz.open(PDF_PATH)
    TOTAL_PAGES = pdf.page_count
    pdf.close()

    os.makedirs(f"vector_dbs/{PDF_NAME}", exist_ok=True)

    return (
        f"✅ '{PDF_NAME}' loaded ({TOTAL_PAGES} pages).",
        "",
        gr.update(value=0, minimum=0, maximum=max(0, TOTAL_PAGES - 1), step=1)
    )


# ---------------------------------------------------------
#  Update page display
# ---------------------------------------------------------
def update_page(page_num):
    try:
        page_num = int(page_num)
    except:
        return "⚠️ Invalid page index.", ""

    store, text = get_or_create_page_vector(page_num)

    if store is None and text is None:
        return "⚠️ No PDF loaded.", ""

    return f"📖 Page {page_num+1} ready.", text or ""


def update_page_with_highlight(page_num, matched_chunk):
    status, text = update_page(page_num)
    highlighted = highlight_text(text, matched_chunk)
    return status, highlighted


# ---------------------------------------------------------
#  Voice query (current page)
# ---------------------------------------------------------
def voice_query(audio, page_num):
    try:
        result = whisper_model.transcribe(audio)
        text = result.get("text", "")

        if WAKE_WORD not in text.lower():
            return f"⚠️ Wake word not detected. Say '{WAKE_WORD}'.", None

        query = text.lower().replace(WAKE_WORD, "").strip()

        answer, best_chunk = chat_with_pdf_current_page(query, int(page_num))

        output_file = "response.mp3"
        if os.path.exists(output_file):
            os.remove(output_file)

        tts = gTTS(answer)
        tts.save(output_file)

        return f"🎤 You said: {query}\n\n🤖 {answer}", output_file

    except Exception as e:
        return f"❌ Error: {e}", None


# ---------------------------------------------------------
#                      GRADIO UI
# ---------------------------------------------------------
with gr.Blocks(theme="soft") as demo:

    gr.Markdown("## 📚 Advanced PDF RAG (Page, Range, Compare, Highlight, Voice)")

    pdf_upload = gr.File(label="📄 Upload PDF")
    status = gr.Textbox(label="Status", interactive=False)
    pdf_text = gr.HTML("<i>Page text will appear here</i>")

    page_slider = gr.Slider(label="Page", minimum=0, maximum=0, step=1, value=0)

    with gr.Row():
        page_from = gr.Number(label="From Page", value=0)
        page_to = gr.Number(label="To Page", value=0)

    question = gr.Textbox(label="Ask Question")
    answer = gr.Textbox(label="Answer", lines=6)
    matched_chunk = gr.Textbox(visible=False)

    ask_current_btn = gr.Button("Ask Current Page")
    ask_range_btn = gr.Button("Ask Page Range")

    with gr.Row():
        compare_page_a = gr.Number(label="Page A", value=0)
        compare_page_b = gr.Number(label="Page B", value=1)
    compare_btn = gr.Button("🔍 Compare Pages")
    compare_output = gr.Textbox(label="Comparison", lines=6)

    audio_in = gr.Audio(label="🎙️ Speak Query", type="filepath")
    audio_out = gr.Audio(label="🔊 Audio Reply")

    # Handlers
    pdf_upload.change(process_pdf, inputs=pdf_upload, outputs=[status, pdf_text, page_slider])
    page_slider.change(update_page_with_highlight, inputs=[page_slider, matched_chunk],
                       outputs=[status, pdf_text])

    ask_current_btn.click(
        lambda q, p: chat_with_pdf_current_page(q, int(p)),
        inputs=[question, page_slider],
        outputs=[answer, matched_chunk]
    )

    ask_range_btn.click(
        lambda q, pf, pt: multi_page_rag(q, pf, pt),
        inputs=[question, page_from, page_to],
        outputs=[answer, matched_chunk]
    )

    compare_btn.click(
        compare_pages,
        inputs=[compare_page_a, compare_page_b],
        outputs=compare_output
    )

    audio_in.change(
        voice_query,
        inputs=[audio_in, page_slider],
        outputs=[answer, audio_out]
    )

demo.launch(share=True, debug=True, inline=False, prevent_thread_lock=True)


OLD CODE

In [ ]:
# --- IMPORTS ---
import fitz
import os
import gradio as gr
import whisper
import soundfile as sf
from gtts import gTTS
import nest_asyncio
nest_asyncio.apply()

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate


# --- CONFIG ---
os.environ["GROQ_API_KEY"] = "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"
HUGGINGFACE_TOKEN = "your_huggingface_token"

# Models
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
embeddings = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

prompt = PromptTemplate(
    template="""
You are a helpful AI assistant that answers based only on the PDF context below.
If you don’t know the answer, say “I don’t know from this page.”

Context: {context}
Question: {question}
""",
    input_variables=["context", "question"]
)


# --- GLOBALS ---
PDF_NAME = None
PDF_PATH = None
TOTAL_PAGES = 0

whisper_model = whisper.load_model("base")
WAKE_WORD = "hey"


# --- FUNCTIONS ---
def process_pdf(file):
    """Fast upload: only register PDF metadata (no text extraction yet)."""
    global PDF_NAME, PDF_PATH, TOTAL_PAGES

    PDF_PATH = file.name
    PDF_NAME = os.path.splitext(os.path.basename(file.name))[0]

    pdf = fitz.open(PDF_PATH)
    TOTAL_PAGES = pdf.page_count
    pdf.close()

    os.makedirs(f"vector_dbs/{PDF_NAME}", exist_ok=True)

    return (
        f"✅ '{PDF_NAME}' loaded ({TOTAL_PAGES} pages). Text & vectors will be created on demand.",
        "",
        gr.update(value=0, minimum=0, maximum=TOTAL_PAGES - 1, step=1)
    )


def get_or_create_page_vector(page_num):
    """Lazy-load text + create vector only when needed."""
    page_dir = f"vector_dbs/{PDF_NAME}/page_{page_num+1}"

    # Load existing vectorstore
    if os.path.exists(page_dir):
        store = FAISS.load_local(page_dir, embeddings, allow_dangerous_deserialization=True)
        pdf = fitz.open(PDF_PATH)
        text = pdf.load_page(page_num).get_text("text").strip()
        pdf.close()
        return store, text

    # Create new vectorstore
    pdf = fitz.open(PDF_PATH)
    page = pdf.load_page(page_num)
    text = page.get_text("text").strip()
    pdf.close()

    if not text:
        return None, "⚠️ Page has no readable text."

    docs = splitter.create_documents([text])
    os.makedirs(page_dir, exist_ok=True)

    store = FAISS.from_documents(docs, embeddings)
    store.save_local(page_dir)

    return store, text


def update_page(page_num):
    """When user turns a page."""
    store, text_or_msg = get_or_create_page_vector(page_num)
    if store is None:
        return text_or_msg, ""
    return f"📖 Ready page {page_num+1} (vector index cached).", text_or_msg


def chat_with_pdf(question, page_num):
    """Answer using current page vectors."""
    store, text_or_msg = get_or_create_page_vector(page_num)
    if not store:
        return text_or_msg or "❌ No text/vector available."

    retriever = store.as_retriever(search_type="mmr", search_kwargs={"k": 3})
    docs = retriever.invoke(question)

    context = "\n\n".join(doc.page_content for doc in docs)
    final_prompt = prompt.invoke({"context": context, "question": question})
    answer = llm.invoke(final_prompt)

    return answer.content


def voice_query(audio, page_num):
    """Voice-based query (wake-word activated)."""
    try:
        audio_data, sr = sf.read(audio)
        result = whisper_model.transcribe(audio)
        text = result["text"]

        if WAKE_WORD not in text.lower():
            return f"⚠️ Wake word not detected. Say '{WAKE_WORD}' to activate.", None

        query = text.lower().replace(WAKE_WORD, "").strip()
        answer = chat_with_pdf(query, page_num)

        output_file = "response.mp3"
        if os.path.exists(output_file):
            os.remove(output_file)

        tts = gTTS(answer)
        tts.save(output_file)

        return f"🎤 You said: {query}\n🤖 Answer: {answer}", output_file

    except Exception as e:
        return f"❌ Error: {e}", None


# --- GRADIO UI ---
with gr.Blocks(theme="soft") as demo:
    gr.Markdown("## 📚 Lazy-Loading Page-wise Voice + Chat RAG Assistant")

    pdf_upload = gr.File(label="📄 Upload PDF")
    status = gr.Textbox(label="Status", interactive=False)
    pdf_text = gr.Textbox(label="Page Text", lines=10)

    page_slider = gr.Slider(label="Page", minimum=0, maximum=1, step=1, value=0)
    question = gr.Textbox(label="Ask Question About Current Page")
    answer = gr.Textbox(label="Answer")

    audio_in = gr.Audio(label="🎙️ Speak Query", type="filepath")
    audio_out = gr.Audio(label="🔊 Audio Reply")

    # Events
    pdf_upload.change(
        fn=process_pdf,
        inputs=pdf_upload,
        outputs=[status, pdf_text, page_slider]
    )

    page_slider.change(
        fn=update_page,
        inputs=page_slider,
        outputs=[status, pdf_text]
    )

    question.submit(
        fn=chat_with_pdf,
        inputs=[question, page_slider],
        outputs=answer
    )

    audio_in.change(
        fn=voice_query,
        inputs=[audio_in, page_slider],
        outputs=[answer, audio_out]
    )

demo.launch(share=True, debug=True, inline=False, prevent_thread_lock=True)


ModuleNotFoundError: No module named 'fitz'